### Initial imports

In [88]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Imputer
import pandas as pd
import numpy as np
import os
import re
from dateutil.relativedelta import relativedelta
from statsmodels.tsa import seasonal
from matplotlib import pyplot as plt
from functools import reduce  # Operação de reduce para cálculo de média de uma lista
from datetime import datetime
import math
%matplotlib inline

#Configurações para exibição de tableas
pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)



height has been deprecated.



### Auxiliary functions

In [89]:
# Nomes das colunas adicionadas ao dataframe
COLUNA_ANOMALIA_ACUMULADA = "anomalia_acumulada"
COLUNA_ANOMALIA_DO_MES = "anomalia_mensal"
COLUNA_MEDIA_MENSAL = "media_mensal"

# Todo mês possui 25 linhas por 38 colunas que dá 950
BLOCO_DE_DADOS_DE_UM_MES = 950
QUANTIDADE_DE_VALORES_DO_ARQUIVO = 573800  # (950 blocos x 604 meses,01/1964 até 04/2014)

class LatitudeColumns:

    def __init__(self):
        self.values = range(29, -21, -2)
        self.positive = 'N'
        self.negative = 'S'

    def get_all_columns(self):
        colum_names = []
        for value in self.values:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(value) + self.negative
            colum_names.append(colum_name)
        return colum_names

    def get_single_column(self, desired_value, human_readable=True):
        sub_values = []
        for value in self.values:
            if value != desired_value:
                continue
            sub_values.append(value)
        if human_readable:
            return self.parse_human_readable(sub_values)
        return sub_values

    def get_range(self, init, final, human_readable=True):
        final, init = self.switch(final, init)

        sub_values = []
        if init == final:
            return self.get_single_column(init, human_readable)

        for value in self.values:
            if (value > init) or (value < final):
                continue
            sub_values.append(value)

        if human_readable:
            return self.parse_human_readable(sub_values)

        return sub_values

    def switch(self, final, init):
        if final > init:
            aux = init
            init = final
            final = aux
        return final, init

    def parse_human_readable(self, list):
        colum_names = []
        for value in list:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(-value) + self.negative
            colum_names.append(colum_name)
        return colum_names

class LongitudeColumns:

    def __init__(self):
        #From map 60W to 15E(-17 because 'force' go to -15)
        self.values = range(59, -17, -2)
        self.positive = 'W'
        self.negative = 'E'

    def get_all_columns(self):
        colum_names = []
        for value in self.values:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(value) + self.negative
            colum_names.append(colum_name)
        return colum_names

    def get_single_column(self, desired_value, human_readable=True):
        sub_values = []
        for value in self.values:
            if value != desired_value:
                continue
            sub_values.append(value)
        if human_readable:
            return self.parse_human_readable(sub_values)
        return sub_values

    def get_range(self, init, final, human_readable=True):
        final, init = self.switch(final, init)

        sub_values = []
        if init == final:
            return self.get_single_column(init, human_readable)

        for value in self.values:
            if (value > init) or (value < final):
                continue
            sub_values.append(value)

        if human_readable:
            return self.parse_human_readable(sub_values)

        return sub_values

    def switch(self, final, init):
        if final > init:
            aux = init
            init = final
            final = aux
        return final, init

    def parse_human_readable(self, list):
        colum_names = []
        for value in list:
            if value > 0:
                colum_name = str(value) + self.positive
            else:
                colum_name = str(-value) + self.negative
            colum_names.append(colum_name)
        return colum_names


def constroi_colunas_latitude_longitude(init_lat=29, end_lat=-21,
                                        init_long=59, end_long=-17):
    lat = LatitudeColumns().get_range(init_lat, end_lat)
    long = LongitudeColumns().get_range(init_long, end_long)
    colunas_do_data_frame = []
    for linha in lat:
        for coluna in long:
            lat_long = linha + "-" + coluna
            colunas_do_data_frame.append(lat_long)
    return colunas_do_data_frame


def carrega_array_com_valores_do_arquivo_geral(
        arquivo_com_decadas_de_anomalia="funceme_db/anomalia_tsm/geral/_Dados_TSMvento_2014_04_anomt6414b04"):
    global QUANTIDADE_DE_VALORES_DO_ARQUIVO

    conteudo_do_arquivo = open(arquivo_com_decadas_de_anomalia).read()
    conteudo_do_arquivo = conteudo_do_arquivo.replace("\n", "")

    # Carrega todos os dados de anomalia em um único array
    qtd_char_no_arquivo = 5
    # Todos os valores do arquivo em um único array. Não há separação de mês. Tudo está de forma sequencial
    valores_do_arquivo = []
    for rows_index in range(QUANTIDADE_DE_VALORES_DO_ARQUIVO):
        # slice data like (n:n+5)
        value = float(conteudo_do_arquivo[
                      rows_index * qtd_char_no_arquivo: rows_index * qtd_char_no_arquivo + qtd_char_no_arquivo])
        value = float("%.3f" % value)
        value /= 10
        valores_do_arquivo.append(value)
    return valores_do_arquivo


def carrega_array_com_valores_do_arquivo_mensal(file_name):
    file_content = open(file_name).read()

    # Remove header de um único arquivo
    file_content = file_content[25:]
    file_content = file_content.replace("\n", "")

    block_size = 5
    dados_do_arquivo = []
    for rows_index in range(BLOCO_DE_DADOS_DE_UM_MES):
        # slice data like (n:n+5)
        value = float(file_content[rows_index * block_size: rows_index * block_size + block_size])
        value = float("%.3f" % value)
        value /= 10
        dados_do_arquivo.append(value)

    return dados_do_arquivo


def merge_dados_do_diretorio(diretorio_arquivo_geral,diretorio_arquivo_individual):
    global QUANTIDADE_DE_VALORES_DO_ARQUIVO
    qtde = QUANTIDADE_DE_VALORES_DO_ARQUIVO
    lista_de_arquivos_individuais = []
    arquivos_do_diretorio = os.listdir(diretorio_arquivo_individual)

    quantidade_de_arquivos = 0
    # Adiciona apenas arquivos com extensão .22
    for arquivo in arquivos_do_diretorio:
        if arquivo.endswith(".22"):
            lista_de_arquivos_individuais.append(arquivo)
            quantidade_de_arquivos += 1

    valores_dos_arquivos = carrega_array_com_valores_do_arquivo_geral(diretorio_arquivo_geral)
    # Para cada arquivo na lista é feito append na lista full_data
    for arquivo in lista_de_arquivos_individuais:
        dados_mensais = carrega_array_com_valores_do_arquivo_mensal(diretorio_arquivo_individual + arquivo)
        for item in dados_mensais:
            valores_dos_arquivos.append(item)

    ##44 meses de 05/2014 até 12/2017
    qtde += quantidade_de_arquivos * BLOCO_DE_DADOS_DE_UM_MES  # Que dá 41800

    array_de_anomalias_por_mes = []
    for i in range(0, qtde, 950):
        anomalias_do_mes = valores_dos_arquivos[i:i + BLOCO_DE_DADOS_DE_UM_MES]
        array_de_anomalias_por_mes.append(anomalias_do_mes)
    return array_de_anomalias_por_mes


def inicia_funceme_data_frame(array_de_anomalias_por_mes):
    funceme_rainy_df = pd.DataFrame()
    for anomalias_do_mes in array_de_anomalias_por_mes:
        data = np.array(anomalias_do_mes)
        row_df = pd.DataFrame(data.reshape(-1, len(data)), columns=constroi_colunas_latitude_longitude())
        funceme_rainy_df = funceme_rainy_df.append(row_df)
    funceme_rainy_df.index = range(0, len(array_de_anomalias_por_mes), 1)
    # ### Setando indices baseados na data
    FORMAT = "%Y-%m"
    some_date_time1 = "1964-01"
    data_inicial = datetime.strptime(some_date_time1, FORMAT)
    indexes_data = []
    for i in range(len(array_de_anomalias_por_mes)):
        indexes_data.append(data_inicial + relativedelta(months=i))
    funceme_rainy_df = funceme_rainy_df.set_index(pd.DatetimeIndex(data=indexes_data))
    
    return funceme_rainy_df

def plota_coluna_do_dataframe(dataframe,titulo, nome_da_coluna, save_figure=False):
    fig, axarr = plt.subplots(1)
    fig.set_size_inches(8, 5)
    ax=dataframe[nome_da_coluna].plot( color='b', linestyle='-', grid=True)
    ax.set(xlabel="Year", ylabel="Celsius/10")

    plt.title(titulo)
    plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=2.0)
    plt.axhline(0, color='black')
    if save_figure:
        plt.savefig("Imagens/tsm/"+titulo)
    else:        
        plt.show()
    
    plt.close()

def adiciona_media_mensal(dataframe):
    # Cria média para cada mês adicionando a coluna "media_mensal"
    media_da_figura_no_mes = []

    for date, row in dataframe.iterrows():
        media = (reduce(lambda x, y: x + y, row) / len(row))
        media_da_figura_no_mes.append(media)

    # Cria nova coluna chamada media_mensal e adiciona ao dataframe da funceme(funceme_rainy_df)
    dataframe.loc[:, "%s" % COLUNA_MEDIA_MENSAL] = pd.Series(media_da_figura_no_mes, index=dataframe.index)
    return dataframe

def calcula_climatologia_para_dataframe(dataframe):
    # Cria dicionário de médias anuais
    medias_anuais = {}
    for mes in range(1, 13, 1):
        medias_anuais[mes] = []

    for date, row in dataframe.iterrows():
        data = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
        medias_anuais[data.month].append(row[('%s' % COLUNA_MEDIA_MENSAL)])

    # Calcula climatologia para cada mês
    climatologias_mensais = {}
    for mes in range(1, 13, 1):
        climatologias_mensais[mes] = reduce(lambda x, y: x + y, medias_anuais[mes]) / len(medias_anuais[mes])
    return climatologias_mensais

def adiciona_anomalia(dataframe):
    climatologias_mensais = calcula_climatologia_para_dataframe(dataframe)

    ### Calculando anomalias
    anomalia = []
    for date, row in dataframe.iterrows():
        data = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
        anomalia_do_mes = row[COLUNA_MEDIA_MENSAL] - climatologias_mensais[data.month]
        anomalia.append(anomalia_do_mes)
    dataframe.loc[:, COLUNA_ANOMALIA_DO_MES] = pd.Series(anomalia, index=dataframe.index)
    dataframe.head(2)
    return dataframe

def adiciona_anomalia_acumulada(dataframe):
    anomalia_acumulada = []
    # Calcula anomalia acumulada
    for index in range(len(dataframe.index)):
        if index == 0:
            anomalia_acumulada.append(dataframe.iloc[index][COLUNA_ANOMALIA_DO_MES])
            continue
        anterior = anomalia_acumulada[index-1]
        atual = dataframe.iloc[index][COLUNA_ANOMALIA_DO_MES]
        anomalia_acumulada.append(float("%.3f" % (atual+anterior)))

    dataframe.loc[:, COLUNA_ANOMALIA_ACUMULADA] = pd.Series(anomalia_acumulada, index=dataframe.index)
    return dataframe

# Analisys starts from here

## Rainfall data from FUNCEME - Scraped since 1973

In [90]:
funceme_rainy_df = pd.read_csv("scraping/funceme_media_macrorregiao.csv", index_col=0 ,parse_dates=['datahora'])
media_observado_a_substituir = funceme_rainy_df.loc['1973-08-01 12:00:00']['Observado(mm)'].mean()
media_desvio_a_substituir = funceme_rainy_df.loc['1973-08-01 12:00:00']['Desvio(%)'].mean()

#Trata missing number
funceme_rainy_df['Normal(mm)'].fillna(media_observado_a_substituir, inplace=True)
funceme_rainy_df['Observado(mm)'].fillna(media_observado_a_substituir, inplace=True)
funceme_rainy_df['Desvio(%)'].fillna(media_desvio_a_substituir, inplace=True)

#Obtém média para cada coluna dos dados pluviométricos
medias = []
observado = []
desvio = []
for indices_unicos in funceme_rainy_df.index.unique():
    medias.append(funceme_rainy_df.loc[indices_unicos]['Normal(mm)'].mean())
    observado.append(funceme_rainy_df.loc[indices_unicos]['Observado(mm)'].mean())
    desvio.append(funceme_rainy_df.loc[indices_unicos]['Desvio(%)'].mean())

    #Cria novo Pandas Dataframe
funceme_rainy_montlhy_df = pd.DataFrame(index=funceme_rainy_df.index.unique().tolist())

#Adiciona dados mensais do estado ao Dataframe criado
funceme_rainy_montlhy_df = pd.concat(
    [funceme_rainy_montlhy_df, 
        pd.DataFrame(data=medias, index=funceme_rainy_montlhy_df.index, columns= ['Normal(mm)']),
        pd.DataFrame(data=observado, index=funceme_rainy_montlhy_df.index, columns= ['Observado(mm)']),
        pd.DataFrame(data=desvio, index=funceme_rainy_montlhy_df.index, columns= ['Desvio(%)'])],
    axis=1, join_axes=[funceme_rainy_montlhy_df.index])
funceme_rainy_montlhy_df.head()

##Set values to class
rainy_seasonal_months = [2,3,4] #February, March, April

indexes = []
rows = []

for index,row in funceme_rainy_montlhy_df.iterrows():
    if index.month not in rainy_seasonal_months:
        continue
    indexes.append(index)
    rows.append(row)

rainy_classified_df = pd.DataFrame(index=indexes, columns=funceme_rainy_montlhy_df.columns,data=rows)

rainy_classified_df = rainy_classified_df.groupby(rainy_classified_df.index.year).mean()

strong = 'strong'
normal = 'normal'
weak = 'weak'

classes = []

for index, row in rainy_classified_df.iterrows():
#     print(index)
    if row['Observado(mm)'] < 200:
        classes.append(weak)
        continue
    
    if row['Observado(mm)'] < 300:
        classes.append(normal)
        continue
        
    classes.append(strong)
rainy_classified_df['classes'] = classes
rainy_classified_df = rainy_classified_df['classes']
rainy_classified_df.head(5)

1973    normal
1974    strong
1975    normal
1976      weak
1977      weak
Name: classes, dtype: object

## Filter SST anomaly from 1973

In [91]:
array_de_anomalias_por_mes = merge_dados_do_diretorio("funceme_db/anomalia_tsm/geral/_Dados_TSMvento_2014_04_anomt6414b04","funceme_db/anomalia_tsm/individual/")
geral_df = inicia_funceme_data_frame(array_de_anomalias_por_mes)
funceme_sst_df = geral_df.loc['1973-01-01':'2017-12-01']
funceme_sst_df = funceme_sst_df.replace(9999.8, np.nan)
funceme_sst_df = funceme_sst_df.dropna(axis=1, how='any')

# Probably won`t use it
# funceme_sst_df = adiciona_media_mensal(funceme_sst_df)
# funceme_sst_df = adiciona_anomalia(funceme_sst_df)
# funceme_sst_df = adiciona_anomalia_acumulada(funceme_sst_df)

## Remove first semester from SST dataframe

In [92]:
second_semester_months=[7,8,9,10,11,12] # From August to December


In [93]:
indexes = []
rows = []

for index,row in funceme_sst_df.iterrows():
    if index.month not in second_semester_months:
        continue
    indexes.append(index)
    rows.append(row)

sst_full_area_jul_dez_df = pd.DataFrame(index=indexes, columns=funceme_sst_df.columns,data=rows)
sst_full_area_jul_dez_df.head(3)

,29N-59W,29N-57W,29N-55W,29N-53W,29N-51W,29N-49W,29N-47W,29N-45W,29N-43W,29N-41W,29N-39W,29N-37W,29N-35W,29N-33W,29N-31W,29N-29W,29N-27W,29N-25W,29N-23W,29N-21W,29N-19W,29N-17W,29N-15W,29N-13W,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,27N-15W,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,25N-15W,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,15N-59W,15N-57W,15N-55W,15N-53W,15N-51W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,15N-19W,15N-17W,13N-59W,13N-57W,13N-55W,13N-53W,13N-51W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,13N-19W,13N-17W,11N-59W,11N-57W,11N-55W,11N-53W,11N-51W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,11N-19W,11N-17W,11N-15W,9N-59W,9N-57W,9N-55W,9N-53W,9N-51W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,9N-19W,9N-17W,9N-15W,9N-13W,7N-57W,...,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,1S-1E,1S-3E,1S-5E,1S-7E,1S-9E,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,3S-1E,3S-3E,3S-5E,3S-7E,3S-9E,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,5S-1E,5S-3E,5S-5E,5S-7E,5S-9E,5S-11E,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E
1973-07-01,-1.1,-1.0,-1.0,-1.1,-1.0,-1.0,-0.8,-0.7,-0.6,-0.4,-0.5,-0.5,-0.5,-0.4,-0.6,-0.6,-0.6,-0.6,-0.7,-0.6,-0.4,-0.4,-0.2,-0.3,-0.6,-0.6,-0.7,-0.8,-0.8,-0.8,-0.7,-0.6,-0.5,-0.6,-0.6,-0.7,-0.7,-0.7,-0.7,-0.6,-0.6,-0.6,-0.6,-0.5,-0.3,-

## Creating transformed df from SST full area

In [94]:
def create_transformed_df(dataframe, column_name_prefix, initial_year = 1973, end_year=2018):
    columns = dataframe.columns 
    months = second_semester_months
    merged_colum_names=[]
    

    for col in columns:
        for month in months:
            column_name = column_name_prefix + col + '_' +str(month)
            merged_colum_names.append(column_name)
            
    dataframe_transformed_df = pd.DataFrame(index=range(1973,2018,1), columns=merged_colum_names)
    
    for year in range(initial_year, end_year, 1):
    #     print("Year " , year) # FOR DEBUG ONLY
        for month in range(7,13,1):
            for column_name in dataframe.columns:
                value = dataframe.loc[str(year)+'-'+str(month)+'-01'][column_name]
                column_name = column_name_prefix +column_name+'_'+str(month)
                dataframe_transformed_df.set_value(year, column_name, value)
    return dataframe_transformed_df

In [95]:
# columns = sst_full_area_jul_dez_df.columns 
# months = second_semester_months
# merged_colum_names=[]

# for col in columns:
#     for month in months:
#         column_name = 'sst_' + col + '_' +str(month)
#         merged_colum_names.append(column_name)

In [96]:
#For debugy only
# for column_name in merged_colum_names:
#     print(column_name)

In [97]:
# sst_transformed_df = pd.DataFrame(index=range(1973,2018,1), columns=merged_colum_names)

In [ ]:
# for year in range(1973,2018,1):
#     #     print("Year " , year) # FOR DEBUG ONLY
#     for month in range(7,13,1):
#         for sst_column in sst_full_area_jul_dez_df.columns:
#             value = sst_full_area_jul_dez_df.loc[str(year)+'-'+str(month)+'-01'][sst_column]
#             column_name = 'sst_' +sst_column+'_'+str(month)
#             sst_transformed_df.set_value(year, column_name, value)
            

In [ ]:
sst_transformed_df = create_transformed_df(sst_full_area_jul_dez_df, "sst_")

In [ ]:
sst_transformed_df.head(3)

In [ ]:
sst_transformed_df.head(3)

## PSW

In [ ]:
array_dados_brutos_x = merge_dados_do_diretorio("funceme_db/pseudo_tensao_x_dados_brutos/geral/_Dados_TSMvento_2014_04_pwsx6414b04","funceme_db/pseudo_tensao_x_dados_brutos/individual/")
array_dados_brutos_y = merge_dados_do_diretorio("funceme_db/pseudo_tensao_y_dados_brutos/geral/_Dados_TSMvento_2014_04_pwsy6414b04","funceme_db/pseudo_tensao_y_dados_brutos/individual/")

In [ ]:
#For debug only
#array_dados_brutos_x[0]

In [ ]:
#For debug only
# array_dados_brutos_y[0]

In [ ]:
pws = []
for month_index in range(len(array_dados_brutos_x)):
    scalar_product_list = []
    for value_index in range(len(array_dados_brutos_x[month_index])):
        scalar_product = math.sqrt(pow(array_dados_brutos_x[month_index][value_index],2) + pow(array_dados_brutos_y[month_index][value_index],2))
        if scalar_product == 14141.852781018475:
            scalar_product_list.append(np.nan)
        else:
            scalar_product_list.append(scalar_product)
    pws.append(scalar_product_list)

In [ ]:
#For debug only
# pws[0][0]

In [ ]:
#For debug only
# pws[0][1]

In [ ]:
pws_full_area_df = inicia_funceme_data_frame(pws)
pws_full_area_df.head(3)

In [ ]:
## Remove first semester from PWS dataframe

In [ ]:
indexes = []
rows = []

for index,row in pws_full_area_df.iterrows():
    if index.month not in second_semester_months:
        continue
    indexes.append(index)
    rows.append(row)

pws_full_area_jul_dez_df = pd.DataFrame(index=indexes, columns=pws_full_area_df.columns,data=rows)


# TESTIN THIS CASE

In [ ]:
pws_full_area_jul_dez_df.head(3)

### Treating missing data

In [ ]:
pws_full_area_jul_dez_df = pws_full_area_jul_dez_df.replace(14141.852781018475, np.nan)
pws_full_area_jul_dez_df.head(3)

In [ ]:
pws_full_area_jul_dez_df = pws_full_area_jul_dez_df.dropna(axis=1, how='any')
pws_full_area_jul_dez_df.head(2)

In [ ]:
## Creating transformed df PWS full area

In [ ]:
columns = pws_full_area_jul_dez_df.columns 
months = second_semester_months
pws_merged_colum_names=[]

for col in columns:
    for month in months:
        column_name = 'pws_'+ col + '_' +str(month)
        pws_merged_colum_names.append(column_name)

        #For debug only        
# for column_name in merged_colum_names:
#     print(column_name)
            

In [ ]:
pws_transformed_df = pd.DataFrame(index=range(1973,2018,1), columns=pws_merged_colum_names)

for year in range(1973,2018,1):
#     print("Year " , year) #For debug only
    for month in range(7,13,1):
        for pws_column in pws_full_area_jul_dez_df.columns:
            value = pws_full_area_jul_dez_df.loc[str(year)+'-'+str(month)+'-01'][pws_column]
            column_name = 'pws_' +pws_column+'_'+str(month)
            pws_transformed_df.set_value(year, column_name, value)

In [ ]:
pws_transformed_df = create_transformed_df(pws_full_area_jul_dez_df, "pws_")

In [ ]:
pws_transformed_df.head()

In [32]:
pws_transformed_df.head()

,pws_29N-59W_7,pws_29N-59W_8,pws_29N-59W_9,pws_29N-59W_10,pws_29N-59W_11,pws_29N-59W_12,pws_29N-57W_7,pws_29N-57W_8,pws_29N-57W_9,pws_29N-57W_10,pws_29N-57W_11,pws_29N-57W_12,pws_29N-55W_7,pws_29N-55W_8,pws_29N-55W_9,pws_29N-55W_10,pws_29N-55W_11,pws_29N-55W_12,pws_29N-53W_7,pws_29N-53W_8,pws_29N-53W_9,pws_29N-53W_10,pws_29N-53W_11,pws_29N-53W_12,pws_29N-51W_7,pws_29N-51W_8,pws_29N-51W_9,pws_29N-51W_10,pws_29N-51W_11,pws_29N-51W_12,pws_29N-49W_7,pws_29N-49W_8,pws_29N-49W_9,pws_29N-49W_10,pws_29N-49W_11,pws_29N-49W_12,pws_29N-47W_7,pws_29N-47W_8,pws_29N-47W_9,pws_29N-47W_10,pws_29N-47W_11,pws_29N-47W_12,pws_29N-45W_7,pws_29N-45W_8,pws_29N-45W_9,pws_29N-45W_10,pws_29N-45W_11,pws_29N-45W_12,pws_29N-43W_7,pws_29N-43W_8,pws_29N-43W_9,pws_29N-43W_10,pws_29N-43W_11,pws_29N-43W_12,pws_29N-41W_7,pws_29N-41W_8,pws_29N-41W_9,pws_29N-41W_10,pws_29N-41W_11,pws_29N-41W_12,pws_29N-39W_7,pws_29N-39W_8,pws_29N-39W_9,pws_29N-39W_10,pws_29N-39W_11,pws_29N-39W_12,pws_29N-37W_7,pws_29N-37W_8,pws_29N-37W_9,pws_29N-37W_10,pws_29N-37W_11,pws_29N-37W_12,pws_29N-35W_7,pws_29N-35W_8,pws_29N-35W_9,pws_29N-35W_10,pws_29N-35W_11,pws_29N-35W_12,pws_29N-33W_7,pws_29N-33W_8,pws_29N-33W_9,pws_29N-33W_10,pws_29N-33W_11,pws_29N-33W_12,pws_29N-31W_7,pws_29N-31W_8,pws_29N-31W_9,pws_29N-31W_10,pws_29N-31W_11,pws_29N-31W_12,pws_29N-29W_7,pws_29N-29W_8,pws_29N-29W_9,pws_29N-29W_10,pws_29N-29W_11,pws_29N-29W_12,pws_29N-27W_7,pws_29N-27W_8,pws_29N-27W_9,pws_29N-27W_10,pws_29N-27W_11,pws_29N-27W_12,pws_29N-25W_7,pws_29N-25W_8,pws_29N-25W_9,pws_29N-25W_10,pws_29N-25W_11,pws_29N-25W_12,pws_29N-23W_7,pws_29N-23W_8,pws_29N-23W_9,pws_29N-23W_10,pws_29N-23W_11,pws_29N-23W_12,pws_29N-21W_7,pws_29N-21W_8,pws_29N-21W_9,pws_29N-21W_10,pws_29N-21W_11,pws_29N-21W_12,pws_29N-19W_7,pws_29N-19W_8,pws_29N-19W_9,pws_29N-19W_10,pws_29N-19W_11,pws_29N-19W_12,pws_29N-17W_7,pws_29N-17W_8,pws_29N-17W_9,pws_29N-17W_10,pws_29N-17W_11,pws_29N-17W_12,pws_29N-15W_7,pws_29N-15W_8,pws_29N-15W_9,pws_29N-15W_10,pws_29N-15W_11,pws_29N-15W_12,pws_29N-13W_7,pws_29N-13W_8,pws_29N-13W_9,pws_29N-13W_10,pws_29N-13W_11,pws_29N-13W_12,pws_27N-59W_7,pws_27N-59W_8,pws_27N-59W_9,pws_27N-59W_10,pws_27N-59W_11,pws_27N-59W_12,pws_27N-57W_7,pws_27N-57W_8,pws_27N-57W_9,pws_27N-57W_10,pws_27N-57W_11,pws_27N-57W_12,pws_27N-55W_7,pws_27N-55W_8,pws_27N-55W_9,pws_27N-55W_10,pws_27N-55W_11,pws_27N-55W_12,pws_27N-53W_7,pws_27N-53W_8,pws_27N-53W_9,pws_27N-53W_10,pws_27N-53W_11,pws_27N-53W_12,pws_27N-51W_7,pws_27N-51W_8,pws_27N-51W_9,pws_27N-51W_10,pws_27N-51W_11,pws_27N-51W_12,pws_27N-49W_7,pws_27N-49W_8,pws_27N-49W_9,pws_27N-49W_10,pws_27N-49W_11,pws_27N-49W_12,pws_27N-47W_7,pws_27N-47W_8,pws_27N-47W_9,pws_27N-47W_10,pws_27N-47W_11,pws_27N-47W_12,pws_27N-45W_7,pws_27N-45W_8,pws_27N-45W_9,pws_27N-45W_10,pws_27N-45W_11,pws_27N-45W_12,pws_27N-43W_7,pws_27N-43W_8,pws_27N-43W_9,pws_27N-43W_10,pws_27N-43W_11,pws_27N-43W_12,pws_27N-41W_7,pws_27N-41W_8,pws_27N-41W_9,pws_27N-41W_10,pws_27N-41W_11,pws_27N-41W_12,pws_27N-39W_7,pws_27N-39W_8,pws_27N-39W_9,pws_27N-39W_10,pws_27N-39W_11,pws_27N-39W_12,pws_27N-37W_7,pws_27N-37W_8,pws_27N-37W_9,pws_27N-37W_10,pws_27N-37W_11,pws_27N-37W_12,pws_27N-35W_7,pws_27N-35W_8,pws_27N-35W_9,pws_27N-35W_10,pws_27N-35W_11,pws_27N-35W_12,pws_27N-33W_7,pws_27N-33W_8,pws_27N-33W_9,pws_27N-33W_10,pws_27N-33W_11,pws_27N-33W_12,pws_27N-31W_7,pws_27N-31W_8,pws_27N-31W_9,pws_27N-31W_10,pws_27N-31W_11,pws_27N-31W_12,pws_27N-29W_7,pws_27N-29W_8,pws_27N-29W_9,pws_27N-29W_10,pws_27N-29W_11,pws_27N-29W_12,pws_27N-27W_7,pws_27N-27W_8,pws_27N-27W_9,pws_27N-27W_10,pws_27N-27W_11,pws_27N-27W_12,pws_27N-25W_7,pws_27N-25W_8,pws_27N-25W_9,pws_27N-25W_10,...,pws_17S-17W_9,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_1

# Merge sst_transformed_df and pws_transformed_df

In [33]:
#For debug only
# sst_transformed_df.head()

In [34]:
#For debug only
# pws_transformed_df.head()

In [35]:
df_new = pd.concat([sst_transformed_df, pws_transformed_df], axis=1)
df_new.head()

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,pws_17S-17W_9,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_1

In [36]:
rainy_classified_df

1973    normal
1974    strong
1975    normal
1976      weak
1977      weak
1978      weak
1979      weak
1980      weak
1981      weak
1982      weak
1983      weak
1984    normal
1985    strong
1986    strong
1987      weak
1988    normal
1989    normal
1990      weak
1991      weak
1992      weak
1993      weak
1994    normal
1995    normal
1996    normal
1997      weak
1998      weak
1999      weak
2000    normal
2001      weak
2002      weak
2003    normal
2004      weak
2005      weak
2006      weak
2007      weak
2008    normal
2009    normal
2010      weak
2011      weak
2012      weak
2013      weak
2014      weak
2015      weak
2016      weak
2017      weak
Name: classes, dtype: object

### Droping first row


In [37]:
rainy_classified_df.drop(1973, inplace=True)

In [38]:
rainy_classified_df

1974    strong
1975    normal
1976      weak
1977      weak
1978      weak
1979      weak
1980      weak
1981      weak
1982      weak
1983      weak
1984    normal
1985    strong
1986    strong
1987      weak
1988    normal
1989    normal
1990      weak
1991      weak
1992      weak
1993      weak
1994    normal
1995    normal
1996    normal
1997      weak
1998      weak
1999      weak
2000    normal
2001      weak
2002      weak
2003    normal
2004      weak
2005      weak
2006      weak
2007      weak
2008    normal
2009    normal
2010      weak
2011      weak
2012      weak
2013      weak
2014      weak
2015      weak
2016      weak
2017      weak
Name: classes, dtype: object

# Decrease index to make easy join by index(year)

In [39]:
rainy_classified_df.index = range(1973,2017)

In [40]:
rainy_classified_df

1973    strong
1974    normal
1975      weak
1976      weak
1977      weak
1978      weak
1979      weak
1980      weak
1981      weak
1982      weak
1983    normal
1984    strong
1985    strong
1986      weak
1987    normal
1988    normal
1989      weak
1990      weak
1991      weak
1992      weak
1993    normal
1994    normal
1995    normal
1996      weak
1997      weak
1998      weak
1999    normal
2000      weak
2001      weak
2002    normal
2003      weak
2004      weak
2005      weak
2006      weak
2007    normal
2008    normal
2009      weak
2010      weak
2011      weak
2012      weak
2013      weak
2014      weak
2015      weak
2016      weak
Name: classes, dtype: object

## Merge input and output

In [41]:
dataset_df = pd.concat([df_new, rainy_classified_df], axis=1)
dataset_df.head()

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_10,pws_17S-9W_1

In [42]:
dataset_df.tail()

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_10,pws_17S-9W_1

# Split data, train and predict

## Set train data and test data

### Set train data

In [43]:
#Removing 2017 due we don't have full 2018 available
dataset_df = dataset_df.loc['1973-01-01':'2016-12-01']

#Train
train_df = dataset_df.loc['1973-01-01':'2000-12-01']

### Set test data

In [44]:
#Test
test_df = dataset_df.loc['2000-01-01':'2016-12-01']

In [45]:
test_df.head()

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_10,pws_17S-9W_1

In [46]:
train_df.tail()

,sst_29N-59W_7,sst_29N-59W_8,sst_29N-59W_9,sst_29N-59W_10,sst_29N-59W_11,sst_29N-59W_12,sst_29N-57W_7,sst_29N-57W_8,sst_29N-57W_9,sst_29N-57W_10,sst_29N-57W_11,sst_29N-57W_12,sst_29N-55W_7,sst_29N-55W_8,sst_29N-55W_9,sst_29N-55W_10,sst_29N-55W_11,sst_29N-55W_12,sst_29N-53W_7,sst_29N-53W_8,sst_29N-53W_9,sst_29N-53W_10,sst_29N-53W_11,sst_29N-53W_12,sst_29N-51W_7,sst_29N-51W_8,sst_29N-51W_9,sst_29N-51W_10,sst_29N-51W_11,sst_29N-51W_12,sst_29N-49W_7,sst_29N-49W_8,sst_29N-49W_9,sst_29N-49W_10,sst_29N-49W_11,sst_29N-49W_12,sst_29N-47W_7,sst_29N-47W_8,sst_29N-47W_9,sst_29N-47W_10,sst_29N-47W_11,sst_29N-47W_12,sst_29N-45W_7,sst_29N-45W_8,sst_29N-45W_9,sst_29N-45W_10,sst_29N-45W_11,sst_29N-45W_12,sst_29N-43W_7,sst_29N-43W_8,sst_29N-43W_9,sst_29N-43W_10,sst_29N-43W_11,sst_29N-43W_12,sst_29N-41W_7,sst_29N-41W_8,sst_29N-41W_9,sst_29N-41W_10,sst_29N-41W_11,sst_29N-41W_12,sst_29N-39W_7,sst_29N-39W_8,sst_29N-39W_9,sst_29N-39W_10,sst_29N-39W_11,sst_29N-39W_12,sst_29N-37W_7,sst_29N-37W_8,sst_29N-37W_9,sst_29N-37W_10,sst_29N-37W_11,sst_29N-37W_12,sst_29N-35W_7,sst_29N-35W_8,sst_29N-35W_9,sst_29N-35W_10,sst_29N-35W_11,sst_29N-35W_12,sst_29N-33W_7,sst_29N-33W_8,sst_29N-33W_9,sst_29N-33W_10,sst_29N-33W_11,sst_29N-33W_12,sst_29N-31W_7,sst_29N-31W_8,sst_29N-31W_9,sst_29N-31W_10,sst_29N-31W_11,sst_29N-31W_12,sst_29N-29W_7,sst_29N-29W_8,sst_29N-29W_9,sst_29N-29W_10,sst_29N-29W_11,sst_29N-29W_12,sst_29N-27W_7,sst_29N-27W_8,sst_29N-27W_9,sst_29N-27W_10,sst_29N-27W_11,sst_29N-27W_12,sst_29N-25W_7,sst_29N-25W_8,sst_29N-25W_9,sst_29N-25W_10,sst_29N-25W_11,sst_29N-25W_12,sst_29N-23W_7,sst_29N-23W_8,sst_29N-23W_9,sst_29N-23W_10,sst_29N-23W_11,sst_29N-23W_12,sst_29N-21W_7,sst_29N-21W_8,sst_29N-21W_9,sst_29N-21W_10,sst_29N-21W_11,sst_29N-21W_12,sst_29N-19W_7,sst_29N-19W_8,sst_29N-19W_9,sst_29N-19W_10,sst_29N-19W_11,sst_29N-19W_12,sst_29N-17W_7,sst_29N-17W_8,sst_29N-17W_9,sst_29N-17W_10,sst_29N-17W_11,sst_29N-17W_12,sst_29N-15W_7,sst_29N-15W_8,sst_29N-15W_9,sst_29N-15W_10,sst_29N-15W_11,sst_29N-15W_12,sst_29N-13W_7,sst_29N-13W_8,sst_29N-13W_9,sst_29N-13W_10,sst_29N-13W_11,sst_29N-13W_12,sst_27N-59W_7,sst_27N-59W_8,sst_27N-59W_9,sst_27N-59W_10,sst_27N-59W_11,sst_27N-59W_12,sst_27N-57W_7,sst_27N-57W_8,sst_27N-57W_9,sst_27N-57W_10,sst_27N-57W_11,sst_27N-57W_12,sst_27N-55W_7,sst_27N-55W_8,sst_27N-55W_9,sst_27N-55W_10,sst_27N-55W_11,sst_27N-55W_12,sst_27N-53W_7,sst_27N-53W_8,sst_27N-53W_9,sst_27N-53W_10,sst_27N-53W_11,sst_27N-53W_12,sst_27N-51W_7,sst_27N-51W_8,sst_27N-51W_9,sst_27N-51W_10,sst_27N-51W_11,sst_27N-51W_12,sst_27N-49W_7,sst_27N-49W_8,sst_27N-49W_9,sst_27N-49W_10,sst_27N-49W_11,sst_27N-49W_12,sst_27N-47W_7,sst_27N-47W_8,sst_27N-47W_9,sst_27N-47W_10,sst_27N-47W_11,sst_27N-47W_12,sst_27N-45W_7,sst_27N-45W_8,sst_27N-45W_9,sst_27N-45W_10,sst_27N-45W_11,sst_27N-45W_12,sst_27N-43W_7,sst_27N-43W_8,sst_27N-43W_9,sst_27N-43W_10,sst_27N-43W_11,sst_27N-43W_12,sst_27N-41W_7,sst_27N-41W_8,sst_27N-41W_9,sst_27N-41W_10,sst_27N-41W_11,sst_27N-41W_12,sst_27N-39W_7,sst_27N-39W_8,sst_27N-39W_9,sst_27N-39W_10,sst_27N-39W_11,sst_27N-39W_12,sst_27N-37W_7,sst_27N-37W_8,sst_27N-37W_9,sst_27N-37W_10,sst_27N-37W_11,sst_27N-37W_12,sst_27N-35W_7,sst_27N-35W_8,sst_27N-35W_9,sst_27N-35W_10,sst_27N-35W_11,sst_27N-35W_12,sst_27N-33W_7,sst_27N-33W_8,sst_27N-33W_9,sst_27N-33W_10,sst_27N-33W_11,sst_27N-33W_12,sst_27N-31W_7,sst_27N-31W_8,sst_27N-31W_9,sst_27N-31W_10,sst_27N-31W_11,sst_27N-31W_12,sst_27N-29W_7,sst_27N-29W_8,sst_27N-29W_9,sst_27N-29W_10,sst_27N-29W_11,sst_27N-29W_12,sst_27N-27W_7,sst_27N-27W_8,sst_27N-27W_9,sst_27N-27W_10,sst_27N-27W_11,sst_27N-27W_12,sst_27N-25W_7,sst_27N-25W_8,sst_27N-25W_9,sst_27N-25W_10,...,pws_17S-17W_10,pws_17S-17W_11,pws_17S-17W_12,pws_17S-15W_7,pws_17S-15W_8,pws_17S-15W_9,pws_17S-15W_10,pws_17S-15W_11,pws_17S-15W_12,pws_17S-13W_7,pws_17S-13W_8,pws_17S-13W_9,pws_17S-13W_10,pws_17S-13W_11,pws_17S-13W_12,pws_17S-11W_7,pws_17S-11W_8,pws_17S-11W_9,pws_17S-11W_10,pws_17S-11W_11,pws_17S-11W_12,pws_17S-9W_7,pws_17S-9W_8,pws_17S-9W_9,pws_17S-9W_10,pws_17S-9W_1

### Keep last column(classes) as target

In [47]:
feature_cols = dataset_df.columns
feature_cols=feature_cols.drop('classes')

In [48]:
feature_cols

Index(['sst_29N-59W_7', 'sst_29N-59W_8', 'sst_29N-59W_9', 'sst_29N-59W_10', 'sst_29N-59W_11', 'sst_29N-59W_12', 'sst_29N-57W_7', 'sst_29N-57W_8', 'sst_29N-57W_9', 'sst_29N-57W_10',
       ...
       'pws_19S-11E_9', 'pws_19S-11E_10', 'pws_19S-11E_11', 'pws_19S-11E_12', 'pws_19S-13E_7', 'pws_19S-13E_8', 'pws_19S-13E_9', 'pws_19S-13E_10', 'pws_19S-13E_11', 'pws_19S-13E_12'], dtype='object', length=7536)

### Input data (X)

In [49]:
X = train_df.loc[:, feature_cols]

### Output data (y)

In [50]:
y = train_df['classes']

### Testing

In [51]:
X_test = test_df.loc[:, feature_cols]


In [52]:
X_test.shape


(16, 7536)

# Applyng classifiers

## Testing KNN (with k=3)

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
knn = KNeighborsClassifier(n_neighbors=2)

knn.fit(X, y)
new_pred_class = knn.predict(X_test)
metrics.accuracy_score(test_df['classes'], new_pred_class)
year = 2001
for v in new_pred_class:
    print(year, v)

2001 normal
2001 normal
2001 weak
2001 normal
2001 weak
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 normal
2001 weak
2001 normal
2001 normal
2001 normal
2001 normal


In [54]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.375

In [55]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(X, y)
new_pred_class = svc.predict(X_test)
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [56]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.8125

### GaussianProcessClassifier

In [57]:
from sklearn.gaussian_process import GaussianProcessClassifier

In [58]:
gpc = GaussianProcessClassifier()

In [59]:
gpc.fit(X, y)

GaussianProcessClassifier(copy_X_train=True, kernel=None,
             max_iter_predict=100, multi_class='one_vs_rest', n_jobs=1,
             n_restarts_optimizer=0, optimizer='fmin_l_bfgs_b',
             random_state=None, warm_start=False)

In [60]:
new_pred_class = gpc.predict(X_test)

In [61]:
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [62]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.8125

### Decision Tree

In [63]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier()
dtc.fit(X, y)
new_pred_class = dtc.predict(X_test)
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 normal
2014 weak
2015 normal
2016 weak


In [64]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.6875

### RandomForestClassifier

In [65]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier()
rfc.fit(X, y)
new_pred_class = rfc.predict(X_test)
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 weak
2002 normal
2003 normal
2004 normal
2005 weak
2006 normal
2007 normal
2008 weak
2009 normal
2010 normal
2011 normal
2012 normal
2013 normal
2014 normal
2015 normal
2016 normal


In [66]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.25

### MLPClassifier

In [67]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
mlp.fit(X, y)
new_pred_class = mlp.predict(X_test)
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 weak


In [68]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.8125

In [69]:
### sklearn.ensemble.AdaBoostClassifier

In [70]:
from sklearn.ensemble import AdaBoostClassifier
adc = AdaBoostClassifier()
adc.fit(X, y)
new_pred_class = adc.predict(X_test)
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 normal
2002 weak
2003 weak
2004 weak
2005 weak
2006 normal
2007 weak
2008 weak
2009 weak
2010 weak
2011 weak
2012 weak
2013 weak
2014 weak
2015 normal
2016 weak


In [71]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.625

### GaussianNB

In [72]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X, y)
new_pred_class = gnb.predict(X_test)
year = 2001
for v in new_pred_class:
    print(year, v)
    year += 1

2001 weak
2002 weak
2003 weak
2004 weak
2005 weak
2006 weak
2007 weak
2008 weak
2009 weak
2010 normal
2011 weak
2012 weak
2013 weak
2014 weak
2015 weak
2016 normal


In [73]:
metrics.accuracy_score(test_df['classes'], new_pred_class)

0.6875

### Init columns form regions: ATN, ATS, TN, TS

In [74]:
colunas_atn_n = constroi_colunas_latitude_longitude(init_lat=19, end_lat=0,
                                                    init_long=49, end_long=20)

colunas_atn_s = constroi_colunas_latitude_longitude(init_lat=-1, end_lat=-17,
                                                    init_long=29, end_long=0)

colunas_tn = constroi_colunas_latitude_longitude(init_lat=27, end_lat=3,
                                                    init_long=59, end_long=17)

colunas_ts = constroi_colunas_latitude_longitude(init_lat=5, end_lat=-20,
                                                    init_long=59, end_long=-17)

### Init dataframes from : ATN, ATS, TN, TS

In [75]:
atn_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_atn_n)
ats_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_atn_s)
tn_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_tn)
ts_sst_df=pd.DataFrame(sst_full_area_jul_dez_df, index=sst_full_area_jul_dez_df.index, columns=colunas_ts)

atn_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_atn_n)
ats_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_atn_s)
tn_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_tn)
ts_pws_df=pd.DataFrame(pws_full_area_jul_dez_df, index=pws_full_area_jul_dez_df.index, columns=colunas_ts)

In [76]:
atn_sst_df.head()

,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,7N-49W,7N-47W,7N-45W,7N-43W,7N-41W,7N-39W,7N-37W,7N-35W,7N-33W,7N-31W,7N-29W,7N-27W,7N-25W,7N-23W,7N-21W,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,1N-49W,1N-47W,1N-45W,1N-43W,1N-41W,1N-39W,1N-37W,1N-35W,1N-33W,1N-31W,1N-29W,1N-27W,1N-25W,1N-23W,1N-21W
1973-07-01,-0.2,-0.5,-0.4,-0.3,-0.3,-0.2,-0.2,-0.3,-0.3,-0.3,-0.1,-0.1,-0.2,-0.1,0.0,-0.3,-0.6,-0.4,-0.3,0.0,0.1,0.1,0.1,0.0,-0.2,-0.2,-0.3,-0.2,-0.2,0.2,-0.5,-0.5,-0.5,-0.2,0.1,0.1,0.0,0.1,0.0,-0.1,-0.1,-0.1,0.0,0.0,0.2,-0.5,-0.4,-0.3,-0.1,-0.1,-0.2,-0.3,-0.1,0.0,0.1,0.1,0.2,0.1,0.2,0.2,-0.4,-0.2,0.0,0.2,-0.1,-0.2,-0.4,-0.3,-0.1,0.1,0.2,0.2,0.3,0.4,0.5,-0.3,-0.1,0.1,0.3,0.3,0.1,-0.1,0.0,0.0,0.0,0.1,0.2,0.4,0.6,0.6,-0.2,-0.1,0.0,0.3,0.5,0.5,0.5,0.3,0.2,0.1,0.0,0.1,0.4,0.7,0.8,-0.2,-0.1,0.1,0.3,0.5,0.6,0.8,0.5,0.3,0.2,0.1,0.2,0.4,0.6,0.6,0.1,0.2,0.3,0.4,0.6,0.7,0.7,0.5,0.2,0.2,0.1,0.3,0.5,0.5,0.5,0.6,0.7,0.7,0.6,0.6,0.7,0.6,0.4,0.0,-0.1,0.2,0.6,0.6,0.5,0.7
1973-08-01,-0.2,-0.2,-0.3,-0.6,-0.6,-0.6,-0.5,-0.3,-0.2,-0.2,-0.4,-0.6,-0.5,-0.4,-0.4,-0.1,-0.1,-0.2,-0.4,-0.4,-0.4,-0.3,0.0,0.1,0.0,-0.2,-0.4,-0.5,-0.5,-0.3,-0.3,-0.2,-0.2,-0.2,-0.2,-0.1,-0.1,0.1,0.0,0.0,-0.2,-0.3,-0.3,-0.4,-0.3,-0.6,-0.5,-0.3,-0.1,0.0,0.2,0.1,0.1,0.0,-0.2,-0.2,-0.1,-0.2,-0.2,-0.1,-0.6,-0.4,-0.2,0.1,0.3,0.4,0.4,0.3,0.1,-0.1,-0.1,-0.1,0.0,0.1,0.0,-0.3,-0.2,0.1,0.4,0.5,0.7,0.6,0.5,0.3,0.1,-0.1,-0.1,0.0,0.2,0.3,0.0,0.0,0.2,0.5,0.7,0.8,0.8,0.8,0.7,0.5,0.1,-0.1,0.2,0.4,0.4,0.1,0.1,0.3,0.5,0.6,0.7,0.8,1.0,0.9,0.7,0.3,0.1,0.2,0.5,0.4,0.2,0.0,0.3,0.6,0.7,0.6,0.7,0.8,1.0,0.7,0.4,0.3,0.4,0.5,0.3,0.5,0.1,0.5,0.9,0.9,0.8,0.6,0.7,0.8,0.7,0.7,0.8,0.6,0.6,0.6
1973-09-01,-0.5,-0.5,-0.3,-0.2,0.0,0.2,0.1,-0.2,-0.5,-0.7,-0.7,-0.7,-0.4,-0.2,-0.3,-0.4,-0.4,-0.3,-0.2,-0.1,0.0,0.0,-0.1,-0.2,-0.4,-0.5,-0.6,-0.5,-0.3,-0.3,-0.3,-0.3,-0.4,-0.5,-0.4,-0.4,-0.3,-0.2,-0.2,-0.1,-0.4,-0.5,-0.5,-0.3,-0.3,-0.2,-0.3,-0.5,-0.6,-0.7,-0.7,-0.7,-0.5,-0.3,-0.2,-0.4,-0.4,-0.4,-0.3,-0.3,-0.1,-0.2,-0.4,-0.6,-0.8,-0.7,-0.8,-0.6,-0.4,-0.3,-0.4,-0.4,-0.4,-0.3,-0.2,0.1,0.0,-0.1,-0.1,-0.3,-0.3,-0.3,-0.2,-0.2,-0.2,-0.2,-0.4,-0.3,-0.2,-0.2,0.2,0.2,0.2,0.2,0.2,0.2,0.4,0.3,0.2,0.1,-0.1,-0.2,-0.1,-0.1,-0.1,0.3,0.6,0.6,0.5,0.5,0.6,0.7,0.7,0.4,0.2,0.0,-0.1,0.0,0.1,0.2,0.9,1.0,0.8,0.6,0.6,0.9,1.0,0.9,0.4,0.2,0.0,0.2,0.3,0.4,0.4,1.5,1.5,1.1,0.8,0.8,1.0,1.2,0.9,0.5,0.2,0.3,0.5,0.4,0.4,0.5
1973-10-01,-0.1,-0.1,0.0,0.0,-0.2,-0.5,-0.7,-0.6,-0.3,-0.2,-0.4,-0.7,-0.9,-0.9,-1.1,-0.3,-0.3,0.0,0.2,0.2,-0.1,-0.3,-0.2,-0.2,-0.3,-0.5,-0.9,-1.1,-1.1,-1.1,-0.5,-0.4,-0.2,0.0,0.1,0.0,-0.1,-0.2,-0.3,-0.4,-0.7,-0.9,-1.0,-1.1,-1.1,-0.6,-0.6,-0.5,-0.4,-0.2,-0.1,-0.1,-0.1,-0.3,-0.5,-0.7,-0.9,-0.9,-1.0,-0.8,-0.5,-0.5,-0.6,-0.7,-0.6,-0.3,-0.1,0.0,-0.1,-0.3,-0.4,-0.6,-0.7,-0.7,-0.5,-0.4,-0.5,-0.6,-0.7,-0.6,-0.4,-0.1,0.1,0.1,0.0,-0.1,-0.3,-0.4,-0.4,-0.3,-0.2,-0.2,-0.3,-0.4,-0.4,-0.3,-0.1,0.1,0.1,0.2,0.1,-0.1,-0.1,-0.3,-0.3,0.0,0.2,0.1,0.0,-0.1,-0.2,-0.2,0.0,0.3,0.4,0.3,0.3,0.2,0.0,-0.2,0.4,0.6,0.6,0.5,0.4,0.3,0.2,0.3,0.4,0.5,0.5,0.6,0.6,0.5,0.3,0.5,0.7,0.9,0.8,0.8,0.8,0.6,0.6,0.6,0.6,0.6,0.6,0.8,0.8,0.8
1973-11-01,-0.5,-0.5,-0.3,-0.1,-0.1,-0.2,-0.4,-0.5,-0.6,-0.8,-0.8,-0.9,-0.8,-0.5,-0.2,-0.4,-0.5,-0.4,-0.2,-0.1,-0.1,-0.2,-0.4,-0.6,-0.6

In [77]:
ats_sst_df.head()

,1S-29W,1S-27W,1S-25W,1S-23W,1S-21W,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W
1973-07-01,0.3,1.0,0.9,0.7,0.7,0.9,0.9,0.7,0.5,-0.1,0.1,0.2,0.5,0.8,0.9,0.5,0.8,0.7,0.5,0.3,0.4,0.6,0.5,0.1,-0.1,0.0,0.0,0.2,0.3,0.6,0.4,0.3,0.0,-0.2,-0.2,-0.2,-0.2,-0.2,-0.1,-0.1,-0.2,-0.2,-0.2,-0.1,0.1,0.2,-0.2,-0.5,-0.7,-0.7,-0.7,-0.7,-0.5,0.2,0.3,-0.3,-0.5,-0.4,-0.3,-0.3,0.1,-0.4,-0.7,-0.9,-1.0,-1.0,-0.9,-0.1,0.5,0.7,0.2,-0.4,-0.3,-0.2,-0.1,0.1,-0.2,-0.5,-0.8,-0.9,-0.9,-0.6,0.1,0.7,0.8,0.3,-0.3,-0.2,0.1,0.2,0.2,0.2,0.0,-0.2,-0.2,-0.3,-0.1,0.3,0.6,0.6,0.3,0.0,0.2,0.4,0.4,0.5,0.5,0.6,0.5,0.5,0.5,0.5,0.2,0.1,0.0,0.1,0.3,0.5,0.7,0.5,0.9,1.0,1.0,1.2,1.1,1.0,0.8,0.4,0.0,0.0,0.3,0.7,0.8,0.8,0.7
1973-08-01,0.9,1.0,0.9,0.7,0.8,0.9,1.1,0.9,0.8,0.7,0.8,0.7,0.5,0.3,0.3,0.8,0.9,0.9,0.6,0.5,0.7,0.8,0.9,0.9,0.8,0.7,0.6,0.4,0.2,0.1,0.6,0.4,0.4,0.3,0.1,0.1,0.1,0.4,0.7,0.8,0.6,0.4,0.3,0.2,0.1,0.4,0.2,0.1,0.2,-0.1,-0.3,-0.3,0.0,0.2,0.4,0.4,0.2,0.1,0.1,0.2,0.3,0.1,0.2,0.1,0.1,-0.1,-0.3,-0.2,-0.1,-0.1,-0.1,0.0,0.0,0.0,0.1,0.3,0.2,0.2,0.3,0.3,0.3,0.4,0.0,-0.5,-0.5,-0.3,-0.1,0.0,0.1,0.2,0.6,0.3,0.3,0.4,0.7,0.8,0.9,0.5,-0.2,-0.4,-0.3,0.0,0.2,0.2,0.3,1.0,0.7,0.6,0.7,1.0,1.3,1.4,0.9,0.3,0.0,0.0,0.2,0.2,0.3,0.5,1.3,1.1,1.1,1.2,1.4,1.5,1.5,1.1,0.6,0.3,0.3,0.3,0.4,0.4,0.5
1973-09-01,0.6,0.7,0.6,0.4,0.3,0.5,0.6,0.6,0.5,0.5,0.6,0.5,0.2,0.0,0.2,0.9,1.1,0.7,0.3,0.4,0.6,0.8,0.6,0.3,0.3,0.5,0.5,0.3,0.1,0.2,0.8,1.0,0.7,0.4,0.5,0.8,1.0,0.6,0.2,0.2,0.3,0.5,0.4,0.4,0.4,0.6,0.8,0.6,0.4,0.5,0.9,1.1,0.8,0.3,0.2,0.1,0.1,0.2,0.4,0.4,0.7,0.8,0.6,0.5,0.5,0.8,1.0,0.8,0.5,0.3,0.0,0.1,0.1,0.2,0.3,0.6,0.9,0.7,0.6,0.6,1.0,1.1,1.0,0.8,0.5,0.4,0.3,0.3,0.3,0.5,0.6,1.1,0.9,0.7,0.8,1.0,1.4,1.2,0.9,0.8,0.9,0.9,0.8,0.7,0.6,1.0,1.4,1.1,1.0,1.0,1.1,1.3,1.2,0.9,1.0,1.1,1.2,1.1,1.0,0.8,1.3,1.4,1.3,1.2,1.1,1.1,1.2,1.1,1.0,1.0,1.1,1.2,1.2,1.2,1.0
1973-10-01,0.5,0.6,0.7,0.8,1.0,0.9,0.7,0.5,0.3,0.1,0.4,0.6,0.8,0.9,0.9,0.6,0.6,0.7,0.8,0.8,0.9,0.7,0.4,0.1,0.2,0.5,0.8,0.8,0.8,0.7,0.7,0.7,0.7,0.8,0.8,0.9,0.8,0.3,-0.1,-0.1,0.3,0.9,0.7,0.6,0.3,0.7,0.7,0.6,0.8,0.9,1.1,0.9,0.3,-0.3,-0.4,0.1,0.5,0.5,0.3,0.2,0.7,0.4,0.5,0.7,0.8,0.9,0.8,0.3,-0.2,-0.5,-0.3,0.1,0.1,0.2,0.1,0.7,0.2,0.3,0.4,0.6,0.7,0.7,0.4,0.0,-0.2,-0.3,-0.3,-0.2,0.0,0.1,0.6,0.4,0.2,0.4,0.5,0.6,0.8,0.6,0.4,0.2,0.1,0.0,-0.1,-0.1,-0.1,0.9,0.6,0.5,0.5,0.6,0.7,1.0,1.0,0.8,0.6,0.5,0.3,0.1,0.0,0.0,1.0,0.9,0.7,0.8,0.8,1.0,1.2,1.2,1.1,0.9,0.7,0.6,0.4,0.2,0.3
1973-11-01,-0.3,-0.1,0.1,0.5,0.6,0.4,0.1,0.2,0.3,0.3,0.5,0.5,0.5,0.6,0.4,0.1,0.1,0.4,0.4,0.7,0.8,0.8,0.3,-0.3,-0.3,0.1,0.5,0.5,0.5,0.4,0.5,0.4,0.3,0.2,0.4,0.9,1.1,0.6,-0.1,-0.5,0.0,0.3,0.3,0.4,0.2,0.6,0.3,0.2,0.1,0.3,0.8,1.1,0.9,0.4,0.3,0.3,0.2,0.2,0.2,0.2,0.6,0.6,0.3,0.1,0.2,0.5,0.9,1.2,1.3,1.1,0.8,0.3,0.1,0.1,0.2,0.8,0.9,0.7,0.4,0.2,0.4,0.8,1.2,1.7,1.7,1.3,0.7,0.3,0.2,0.2,0.8,1.2,0.9,0.6,0.3,0.4,0.7,1.2,1.6,1.8,1.5,1.1,0.7,0.4,0.0,0.7,1.0,0.9,0.5,0.3,0.4,0.7,0.9,1.2,1.4,1.5,1.4,1.1,0.6,0.3,0.3,0.6,0.4,0.3,0.4,0.5,0.8,0.8,0.8,1.0,1.2,1.4,1.1,0.9,0.7


In [78]:
tn_sst_df.head()

,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,15N-59W,15N-57W,15N-55W,15N-53W,15N-51W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,15N-19W,15N-17W,13N-59W,13N-57W,13N-55W,13N-53W,13N-51W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,13N-19W,13N-17W,11N-59W,11N-57W,11N-55W,11N-53W,11N-51W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,11N-19W,11N-17W,9N-59W,9N-57W,9N-55W,9N-53W,9N-51W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,9N-19W,9N-17W,7N-59W,7N-57W,7N-55W,7N-53W,7N-51W,7N-49W,7N-47W,7N-45W,7N-43W,7N-41W,7N-39W,7N-37W,7N-35W,7N-33W,7N-31W,7N-29W,7N-27W,7N-25W,7N-23W,7N-21W,7N-19W,7N-17W,5N-59W,5N-57W,5N-55W,5N-53W,5N-51W,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,5N-19W,5N-17W,3N-59W,3N-57W,3N-55W,3N-53W,3N-51W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,3N-19W,3N-17W
1973-07-01,-0.6,-0.6,-0.7,-0.8,-0.8,-0.8,-0.7,-0.6,-0.5,-0.6,-0.6,-0.7,-0.7,-0.7,-0.7,-0.6,-0.6,-0.6,-0.6,-0.5,-0.3,-0.3,-0.2,-0.2,-0.4,-0.5,-0.6,-0.6,-0.5,-0.5,-0.7,-0.8,-0.8,-0.9,-0.9,-0.9,-0.7,-0.5,-0.2,-0.3,-0.4,-0.3,-0.2,-0.2,-0.1,0.0,0.0,-0.2,-0.3,-0.3,-0.3,-0.5,-0.7,-0.9,-0.8,-0.9,-0.9,-0.8,-0.5,-0.2,0.0,-0.1,-0.1,-0.2,-0.3,-0.5,0.1,0.1,0.0,-0.1,-0.1,-0.2,-0.3,-0.4,-0.6,-0.7,-0.7,-0.6,-0.6,-0.5,-0.3,-0.1,0.1,0.1,0.1,-0.2,-0.3,-0.3,0.1,0.1,0.2,0.2,0.0,-0.2,-0.5,-0.4,-0.3,-0.3,-0.2,-0.2,-0.3,-0.3,-0.3,-0.1,-0.1,-0.2,-0.1,0.0,0.1,0.2,0.2,0.1,0.2,0.1,-0.1,-0.3,-0.6,-0.4,-0.3,0.0,0.1,0.1,0.1,0.0,-0.2,-0.2,-0.3,-0.2,-0.2,0.2,0.4,0.7,0.2,0.3,0.2,0.0,-0.3,-0.5,-0.5,-0.5,-0.2,0.1,0.1,0.0,0.1,0.0,-0.1,-0.1,-0.1,0.0,0.0,0.2,0.5,0.7,0.2,0.2,0.0,-0.2,-0.4,-0.5,-0.4,-0.3,-0.1,-0.1,-0.2,-0.3,-0.1,0.0,0.1,0.1,0.2,0.1,0.2,0.2,0.4,0.6,0.0,0.0,-0.2,-0.4,-0.5,-0.4,-0.2,0.0,0.2,-0.1,-0.2,-0.4,-0.3,-0.1,0.1,0.2,0.2,0.3,0.4,0.5,0.4,0.4,-0.3,-0.3,-0.4,-0.5,-0.4,-0.3,-0.1,0.1,0.3,0.3,0.1,-0.1,0.0,0.0,0.0,0.1,0.2,0.4,0.6,0.6,0.6,0.4,NaN,-0.5,-0.5,-0.3,-0.2,-0.2,-0.1,0.0,0.3,0.5,0.5,0.5,0.3,0.2,0.1,0.0,0.1,0.4,0.7,0.8,0.5,0.2,NaN,NaN,NaN,NaN,-0.1,-0.2,-0.1,0.1,0.3,0.5,0.6,0.8,0.5,0.3,0.2,0.1,0.2,0.4,0.6,0.6,0.3,0.2,NaN,NaN,NaN,NaN,NaN,0.1,0.2,0.3,0.4,0.6,0.7,0.7,0.5,0.2,0.2,0.1,0.3,0.5,0.5,0.5,0.5,0.6
1973-08-01,-0.3,-0.3,-0.4,-0.6,-0.6,-0.7,-0.8,-0.8,-0.8,-0.9,-1.0,-1.0,-1.0,-1.1,-1.0,-1.0,-1.0,-0.9,-0.6,-0.5,-0.6,-0.7,-0.4,-0.4,-0.4,-0.5,-0.6,-0.7,-0.7,-0.8,-0.9,-1.0,-1.0,-1.0,-1.2,-1.4,-1.4,-1.3,-1.1,-0.8,-0.6,-0.4,-0.6,-0.8,-0.4,-0.3,-0.4,-0.4,-0.5,-0.5,-0.6,-0.7,-1.0,-0.9,-0.9,-0.8,-1.0,-1.2,-1.3,-1.2,-1.1,-0.7,-0.4,-0.4,-0.7,-1.0,-0.3,-0.3,-0.3,-0.2,-0.3,-0.3,-0.5,-0.5,-0.7,-0.8,-0.8,-0.7,-0.7,-0.7,-0.8,-0.9,-0.9,-0.7,-0.4,-0.4,-0.7,-1.0,-0.3,-0.2,-0.2,-0.2,-0.2,-0.2,-0.2,-0.3,-0.6,

In [79]:
ts_sst_df.head()

,5N-59W,5N-57W,5N-55W,5N-53W,5N-51W,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,5N-19W,5N-17W,5N-15W,5N-13W,5N-11W,5N-9W,5N-7W,5N-5W,5N-3W,5N-1W,5N-1E,5N-3E,5N-5E,5N-7E,5N-9E,5N-11E,5N-13E,5N-15E,3N-59W,3N-57W,3N-55W,3N-53W,3N-51W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,3N-19W,3N-17W,3N-15W,3N-13W,3N-11W,3N-9W,3N-7W,3N-5W,3N-3W,3N-1W,3N-1E,3N-3E,3N-5E,3N-7E,3N-9E,3N-11E,3N-13E,3N-15E,1N-59W,1N-57W,1N-55W,1N-53W,1N-51W,1N-49W,1N-47W,1N-45W,1N-43W,1N-41W,1N-39W,1N-37W,1N-35W,1N-33W,1N-31W,1N-29W,1N-27W,1N-25W,1N-23W,1N-21W,1N-19W,1N-17W,1N-15W,1N-13W,1N-11W,1N-9W,1N-7W,1N-5W,1N-3W,1N-1W,1N-1E,1N-3E,1N-5E,1N-7E,1N-9E,1N-11E,1N-13E,1N-15E,1S-59W,1S-57W,1S-55W,1S-53W,1S-51W,1S-49W,1S-47W,1S-45W,1S-43W,1S-41W,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,1S-29W,1S-27W,1S-25W,1S-23W,1S-21W,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,1S-1E,1S-3E,1S-5E,1S-7E,1S-9E,1S-11E,1S-13E,1S-15E,3S-59W,3S-57W,3S-55W,3S-53W,3S-51W,3S-49W,3S-47W,3S-45W,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,3S-1E,3S-3E,3S-5E,3S-7E,3S-9E,3S-11E,3S-13E,3S-15E,5S-59W,5S-57W,5S-55W,5S-53W,5S-51W,5S-49W,5S-47W,5S-45W,5S-43W,5S-41W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,5S-1E,5S-3E,5S-5E,5S-7E,5S-9E,5S-11E,5S-13E,5S-15E,7S-59W,7S-57W,7S-55W,7S-53W,7S-51W,7S-49W,7S-47W,7S-45W,7S-43W,7S-41W,7S-39W,7S-37W,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,7S-15E,9S-59W,9S-57W,9S-55W,9S-53W,9S-51W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,9S-15E,11S-59W,11S-57W,11S-55W,11S-53W,11S-51W,11S-49W,11S-47W,11S-45W,11S-43W,11S-41W,11S-39W,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,11S-15E,13S-59W,13S-57W,13S-55W,13S-53W,13S-51W,13S-49W,13S-47W,13S-45W,13S-43W,13S-41W,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,13S-15E,15S-59W,15S-57W,15S-55W,15S-53W,15S-51W,15S-49W,15S-47W,15S-45W,15S-43W,15S-41W,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,15S-15E,17S-59W,17S-57W,17S-55W,17S-53W,17S-51W,17S-49W,17S-47W,17S-45W,17S-43W,17S-41W,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,17S-13E,17S-15E,19S-59W,19S-57W,19S-55W,19S-53W,19S-51W,19S-49W,19S-47W,19S-45W,19S-43W,19S-41W,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E,19S-15E
1973-07-01,NaN,NaN,NaN,NaN,-0.1,-0.2,-0.1,0.1,0.3,0.5,0.6,0.8,0.5,0.3,0.2,0.1,0.2,0.4,0.6,0.6,0.3,0.2,0.3,0.3,0.4,0.5,0.5,0.5,0.6,0.7,1.1,1.4,1.4,1.3,1.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1,0.2,0.3,0.4,0.6,0.7,0.7,0.5,0.2,0.2,0.1,0.3,0.5,0.5,0.5,0.5,0.6,0.5,0.4,0.4,0.5,0.7,0.9,1.0,1.2,1.3,1.3,1.2,0.9,0.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.6,0.7,0.7,0.6,0.6,0.7,0.6,0.4,0.0,-0.1,0.2,0.6,0.6,0.5,0.7,0.7,0.9,0.7,0.3,0.2,0.3,0.5,0.8,1.0,1.1,1.1,0.7,0.4,0.2,0.0,NaN,NaN,NaN,NaN,NaN,Na

In [80]:
atn_pws_df.head()

,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,7N-49W,7N-47W,7N-45W,7N-43W,7N-41W,7N-39W,7N-37W,7N-35W,7N-33W,7N-31W,7N-29W,7N-27W,7N-25W,7N-23W,7N-21W,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,1N-49W,1N-47W,1N-45W,1N-43W,1N-41W,1N-39W,1N-37W,1N-35W,1N-33W,1N-31W,1N-29W,1N-27W,1N-25W,1N-23W,1N-21W
1964-07-01,67.856098,72.996507,75.961372,74.215901,70.785945,68.379017,64.066372,60.421519,58.495812,56.738787,52.031721,48.100104,43.744371,38.006315,33.670313,76.068456,81.049676,81.334863,75.156171,68.557859,64.349747,59.255464,53.480931,48.766895,45.009888,40.451576,38.612174,35.361702,28.589683,22.496444,79.030247,82.697340,79.477607,70.677083,64.464331,61.417668,56.690387,50.835126,42.479289,33.835484,27.650136,25.785655,23.982702,19.682480,13.717872,70.208974,71.627160,65.774235,58.052735,53.251479,53.180636,51.807818,47.164499,36.889294,25.179357,16.119864,12.442267,10.938464,9.812747,8.080223,49.556533,48.287990,41.893675,36.654740,35.202983,38.608030,40.838340,37.822877,28.400704,15.920113,4.604346,2.570992,6.600758,8.302409,10.435516,26.752383,24.464055,18.317478,13.780058,15.860013,20.620621,24.573360,23.669812,16.763055,9.620811,8.237718,11.458621,14.873466,16.901183,20.758854,13.001538,11.201786,7.446476,4.104875,3.041381,4.916299,9.600000,13.476646,14.807093,16.101242,16.720048,18.792020,21.689859,26.600752,33.179813,10.692988,11.229426,11.280514,12.702756,14.402778,15.250246,17.755281,20.329781,21.781644,20.930361,20.497805,22.013178,24.960970,32.221887,39.658164,15.454773,17.288725,19.159593,22.380572,24.177055,25.437178,26.802985,25.768974,23.726146,20.942302,19.523319,21.760744,25.684431,32.330326,38.438652,22.583401,25.501569,29.279686,32.472142,33.952025,34.342394,32.203882,28.241813,23.583469,21.002143,21.653868,24.932910,28.435541,30.768328,33.378137
1964-08-01,60.848172,62.391025,60.756070,56.928815,54.610713,55.989285,63.029200,66.813921,67.285957,63.770605,56.660127,46.003696,37.835565,34.934367,35.179113,62.400080,63.779777,62.140566,58.263539,56.766539,59.387709,64.843350,67.635789,64.646191,57.502174,47.852377,38.897301,32.086134,26.892378,23.963514,53.048657,54.803011,52.646842,49.370538,49.289350,52.546360,55.193931,56.240288,50.280016,39.636978,29.093126,22.973028,17.960234,13.376472,10.829589,37.697745,38.124270,34.842072,30.758576,29.076623,32.316714,34.845947,34.112314,27.264079,17.608237,8.184131,3.157531,5.656854,8.471718,10.938464,22.240728,21.092416,15.311434,8.246211,5.780138,6.489992,8.964374,9.231468,4.909175,1.868154,9.501579,17.748521,23.915267,26.728636,27.882970,13.031117,10.186756,6.365532,8.286133,13.325539,16.070159,13.645878,11.232987,11.567627,14.491722,20.888753,31.405891,39.624361,43.176498,44.047815,14.091132,14.839474,15.692355,18.826577,24.919470,29.954799,28.319605,24.103319,20.690336,21.569655,27.907884,38.864508,47.737826,52.080131,52.683584,20.447494,23.405341,25.316003,28.708884,33.852769,39.408755,39.197066,33.057828,27.074896,25.700195,30.978218,40.477648,46.802778,50.800886,51.660527,28.675076,32.249031,35.415957,39.940080,46.250189,51.600097,50.464740,41.769846,31.770584,27.070279,29.823648,36.179690,40.240527,42.311464,42.617485,36.338547,41.099392,46.099024

In [81]:
ats_pws_df.head()

,1S-29W,1S-27W,1S-25W,1S-23W,1S-21W,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W
1964-07-01,27.485451,31.030630,34.033219,33.324165,32.128648,31.062840,26.181291,20.321417,13.800362,9.759098,8.405950,10.863241,14.673105,17.850770,21.026174,31.718922,37.110915,39.668753,38.683976,36.627039,36.249138,33.116914,28.180135,21.366329,15.462535,11.759252,11.455130,13.322537,14.637281,16.949926,34.042473,39.462007,40.376354,39.757515,41.962364,46.779910,47.840255,44.462344,36.826621,28.028022,21.959280,18.620687,17.311268,17.224692,17.473408,34.033072,34.220023,32.467984,34.139274,41.011218,50.438577,58.322208,57.031921,49.300101,39.268944,32.739120,27.181060,23.943475,23.762576,23.520417,31.999062,26.913380,22.556817,25.235689,34.488984,46.113339,54.961532,58.180753,53.012546,43.930058,36.186323,32.035137,29.248761,29.427878,29.060110,28.539446,20.922954,16.584631,19.795201,27.932955,37.763739,46.933357,52.853855,49.739019,43.649742,36.472592,32.548425,31.187337,31.542511,31.920056,26.272800,20.554805,18.873262,21.417049,27.582966,33.761665,41.469266,47.468832,46.724833,42.769265,38.239378,33.670313,33.190511,33.650111,33.082926,25.747427,24.387702,25.492156,28.022313,31.085205,34.089881,39.768958,42.940075,44.960538,44.878614,41.667373,38.910153,38.220021,37.845079,36.086701,26.076810,28.778464,32.825752,36.440774,37.977098,37.901187,39.694458,40.920655,45.576310,48.203112,46.723121,45.984780,45.500000,44.381302,41.583891
1964-08-01,25.134041,28.892213,32.244224,31.312138,28.023205,23.140009,19.731447,20.678491,21.145449,19.915070,17.140595,15.601282,16.977927,19.500000,23.858122,31.722705,36.623080,38.626416,35.558403,29.860509,23.306008,22.340322,24.424782,26.708800,27.261878,24.307406,20.023237,17.751901,18.529976,22.094569,44.055647,50.782871,54.061539,49.729267,39.785927,30.376471,29.302730,33.292942,38.550746,40.049969,36.362618,29.891805,23.085277,21.708293,23.700211,56.635325,66.638202,73.968439,70.277166,57.656656,42.441725,37.692705,42.143208,50.117562,55.030264,53.734533,44.308577,33.019388,27.872926,28.428859,64.803858,79.069716,90.069362,88.416571,74.868485,57.910707,47.638850,48.541220,58.942769,70.132803,73.105745,62.674476,49.021832,38.200524,33.165193,65.384478,81.007716,93.142901,95.223369,86.665160,71.924474,60.407450,56.964024,68.576672,83.846586,91.255356,82.553740,68.593659,52.127728,39.317426,59.122246,71.399860,83.895947,91.921543,90.900440,82.448772,72.698624,68.619531,78.243019,90.854444,100.131164,95.762414,84.403791,66.555541,49.273319,46.910660,54.900729,66.282426,79.626943,87.519712,85.306155,80.416727,78.512419,82.709189,88.170857,94.642485,96.904076,91.408588,76.452992,61.511381,36.140006,39.920421,50.236242,66.868229,79.855119,83.925026,82.404369,82.807306,80.767568,79.083374,80.253660,88.031358,90.331888,81.851390,71.112938
1964-09-01,31.367659,34.122280,34.494202,31.541243,30.072745,29.333428,29.375670,29.461331,26.154350,22.014541,19.015783,17.692089,17.922611,20.608736,25.532724,38.997051,43.649513,43.931765,41.438267,38.415752,37.833583,37.792195,37.481729,33.360006,27.176460,20.645581,16.728718,15.754364,16.857639,21.411212,47.2730

In [82]:
tn_pws_df.head()

,27N-59W,27N-57W,27N-55W,27N-53W,27N-51W,27N-49W,27N-47W,27N-45W,27N-43W,27N-41W,27N-39W,27N-37W,27N-35W,27N-33W,27N-31W,27N-29W,27N-27W,27N-25W,27N-23W,27N-21W,27N-19W,27N-17W,25N-59W,25N-57W,25N-55W,25N-53W,25N-51W,25N-49W,25N-47W,25N-45W,25N-43W,25N-41W,25N-39W,25N-37W,25N-35W,25N-33W,25N-31W,25N-29W,25N-27W,25N-25W,25N-23W,25N-21W,25N-19W,25N-17W,23N-59W,23N-57W,23N-55W,23N-53W,23N-51W,23N-49W,23N-47W,23N-45W,23N-43W,23N-41W,23N-39W,23N-37W,23N-35W,23N-33W,23N-31W,23N-29W,23N-27W,23N-25W,23N-23W,23N-21W,23N-19W,23N-17W,21N-59W,21N-57W,21N-55W,21N-53W,21N-51W,21N-49W,21N-47W,21N-45W,21N-43W,21N-41W,21N-39W,21N-37W,21N-35W,21N-33W,21N-31W,21N-29W,21N-27W,21N-25W,21N-23W,21N-21W,21N-19W,21N-17W,19N-59W,19N-57W,19N-55W,19N-53W,19N-51W,19N-49W,19N-47W,19N-45W,19N-43W,19N-41W,19N-39W,19N-37W,19N-35W,19N-33W,19N-31W,19N-29W,19N-27W,19N-25W,19N-23W,19N-21W,19N-19W,19N-17W,17N-59W,17N-57W,17N-55W,17N-53W,17N-51W,17N-49W,17N-47W,17N-45W,17N-43W,17N-41W,17N-39W,17N-37W,17N-35W,17N-33W,17N-31W,17N-29W,17N-27W,17N-25W,17N-23W,17N-21W,17N-19W,17N-17W,15N-59W,15N-57W,15N-55W,15N-53W,15N-51W,15N-49W,15N-47W,15N-45W,15N-43W,15N-41W,15N-39W,15N-37W,15N-35W,15N-33W,15N-31W,15N-29W,15N-27W,15N-25W,15N-23W,15N-21W,15N-19W,15N-17W,13N-59W,13N-57W,13N-55W,13N-53W,13N-51W,13N-49W,13N-47W,13N-45W,13N-43W,13N-41W,13N-39W,13N-37W,13N-35W,13N-33W,13N-31W,13N-29W,13N-27W,13N-25W,13N-23W,13N-21W,13N-19W,13N-17W,11N-59W,11N-57W,11N-55W,11N-53W,11N-51W,11N-49W,11N-47W,11N-45W,11N-43W,11N-41W,11N-39W,11N-37W,11N-35W,11N-33W,11N-31W,11N-29W,11N-27W,11N-25W,11N-23W,11N-21W,11N-19W,11N-17W,9N-59W,9N-57W,9N-55W,9N-53W,9N-51W,9N-49W,9N-47W,9N-45W,9N-43W,9N-41W,9N-39W,9N-37W,9N-35W,9N-33W,9N-31W,9N-29W,9N-27W,9N-25W,9N-23W,9N-21W,9N-19W,9N-17W,7N-59W,7N-57W,7N-55W,7N-53W,7N-51W,7N-49W,7N-47W,7N-45W,7N-43W,7N-41W,7N-39W,7N-37W,7N-35W,7N-33W,7N-31W,7N-29W,7N-27W,7N-25W,7N-23W,7N-21W,7N-19W,7N-17W,5N-59W,5N-57W,5N-55W,5N-53W,5N-51W,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,5N-19W,5N-17W,3N-59W,3N-57W,3N-55W,3N-53W,3N-51W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,3N-19W,3N-17W
1964-07-01,23.576471,23.433523,23.742788,23.436083,23.400855,23.568835,23.385679,24.758837,28.731342,33.217465,38.946373,45.060071,50.732632,56.257000,59.907679,61.729248,57.360003,54.223334,56.180513,61.494146,69.029342,71.331970,31.533633,33.559648,34.628312,35.300142,35.445733,36.168633,37.561683,39.588382,45.743306,51.138342,56.981488,62.658200,65.836920,66.850580,67.544504,65.046599,58.033180,52.559014,55.822934,62.541186,71.157572,75.700264,40.707862,43.601835,44.700112,45.508900,46.432101,48.027700,51.736641,54.282594,60.096589,66.296606,70.677083,73.594361,74.349714,72.782759,68.831533,65.816715,57.495130,50.806791,52.517807,58.223191,65.048059,70.698303,50.976956,51.618989,51.938521,52.317206,54.553460,58.084852,63.182039,66.456076,68.810246,71.877952,73.438478,71.846016,70.545446,68.597740,64.765577,60.742489,53.811244,48.492680,46.058441,47.181564,50.016397,52.013268,60.851623,57.801903,56.839159,57.742965,61.670901,67.856098,72.996507,75.961372,74.215901,70.785945,68.379017,64.066372,60.421519,58.495812,56.738787,52.031721,48.100104,43.744371,38.006315,33.670313,31.902508,30.870698,66.680432,61.058005,58.702044,61.056777,67.480738,76.068456,81.049676,81.334863,75.156171,68.557859,64.349747,59.255464,53.480931,48.766895,45.009888,40.451576,38.612174,35.361702,28.589683,22.496444,18.269373,17.262677,64.103822,59.073514,56.172947,60.177487,68.790479,79.030247,82.697340,79.477607,70.677083,64.464331,61.417668,56.690387,50.835126,42.479289,33.835484,27.650136,25.785655,23.982702,19.682480,13.717872,10.555094,12.773801,52.199329,48.863586,46.714880,51.084440,60.037655,70.208974,71.627160,65.774235,58.052735,53.251479,53.180636,51.807818,47.164499,36.889294,25.179357,16.119864,12.442267,10.938464,9.812747,8.080223,9.918165,14.280406,33.340216,31.243079,30.739063,3

In [83]:
ts_pws_df.head()

,5N-59W,5N-57W,5N-55W,5N-53W,5N-51W,5N-49W,5N-47W,5N-45W,5N-43W,5N-41W,5N-39W,5N-37W,5N-35W,5N-33W,5N-31W,5N-29W,5N-27W,5N-25W,5N-23W,5N-21W,5N-19W,5N-17W,5N-15W,5N-13W,5N-11W,5N-9W,5N-7W,5N-5W,5N-3W,5N-1W,5N-1E,5N-3E,5N-5E,5N-7E,5N-9E,5N-11E,5N-13E,5N-15E,3N-59W,3N-57W,3N-55W,3N-53W,3N-51W,3N-49W,3N-47W,3N-45W,3N-43W,3N-41W,3N-39W,3N-37W,3N-35W,3N-33W,3N-31W,3N-29W,3N-27W,3N-25W,3N-23W,3N-21W,3N-19W,3N-17W,3N-15W,3N-13W,3N-11W,3N-9W,3N-7W,3N-5W,3N-3W,3N-1W,3N-1E,3N-3E,3N-5E,3N-7E,3N-9E,3N-11E,3N-13E,3N-15E,1N-59W,1N-57W,1N-55W,1N-53W,1N-51W,1N-49W,1N-47W,1N-45W,1N-43W,1N-41W,1N-39W,1N-37W,1N-35W,1N-33W,1N-31W,1N-29W,1N-27W,1N-25W,1N-23W,1N-21W,1N-19W,1N-17W,1N-15W,1N-13W,1N-11W,1N-9W,1N-7W,1N-5W,1N-3W,1N-1W,1N-1E,1N-3E,1N-5E,1N-7E,1N-9E,1N-11E,1N-13E,1N-15E,1S-59W,1S-57W,1S-55W,1S-53W,1S-51W,1S-49W,1S-47W,1S-45W,1S-43W,1S-41W,1S-39W,1S-37W,1S-35W,1S-33W,1S-31W,1S-29W,1S-27W,1S-25W,1S-23W,1S-21W,1S-19W,1S-17W,1S-15W,1S-13W,1S-11W,1S-9W,1S-7W,1S-5W,1S-3W,1S-1W,1S-1E,1S-3E,1S-5E,1S-7E,1S-9E,1S-11E,1S-13E,1S-15E,3S-59W,3S-57W,3S-55W,3S-53W,3S-51W,3S-49W,3S-47W,3S-45W,3S-43W,3S-41W,3S-39W,3S-37W,3S-35W,3S-33W,3S-31W,3S-29W,3S-27W,3S-25W,3S-23W,3S-21W,3S-19W,3S-17W,3S-15W,3S-13W,3S-11W,3S-9W,3S-7W,3S-5W,3S-3W,3S-1W,3S-1E,3S-3E,3S-5E,3S-7E,3S-9E,3S-11E,3S-13E,3S-15E,5S-59W,5S-57W,5S-55W,5S-53W,5S-51W,5S-49W,5S-47W,5S-45W,5S-43W,5S-41W,5S-39W,5S-37W,5S-35W,5S-33W,5S-31W,5S-29W,5S-27W,5S-25W,5S-23W,5S-21W,5S-19W,5S-17W,5S-15W,5S-13W,5S-11W,5S-9W,5S-7W,5S-5W,5S-3W,5S-1W,5S-1E,5S-3E,5S-5E,5S-7E,5S-9E,5S-11E,5S-13E,5S-15E,7S-59W,7S-57W,7S-55W,7S-53W,7S-51W,7S-49W,7S-47W,7S-45W,7S-43W,7S-41W,7S-39W,7S-37W,7S-35W,7S-33W,7S-31W,7S-29W,7S-27W,7S-25W,7S-23W,7S-21W,7S-19W,7S-17W,7S-15W,7S-13W,7S-11W,7S-9W,7S-7W,7S-5W,7S-3W,7S-1W,7S-1E,7S-3E,7S-5E,7S-7E,7S-9E,7S-11E,7S-13E,7S-15E,9S-59W,9S-57W,9S-55W,9S-53W,9S-51W,9S-49W,9S-47W,9S-45W,9S-43W,9S-41W,9S-39W,9S-37W,9S-35W,9S-33W,9S-31W,9S-29W,9S-27W,9S-25W,9S-23W,9S-21W,9S-19W,9S-17W,9S-15W,9S-13W,9S-11W,9S-9W,9S-7W,9S-5W,9S-3W,9S-1W,9S-1E,9S-3E,9S-5E,9S-7E,9S-9E,9S-11E,9S-13E,9S-15E,11S-59W,11S-57W,11S-55W,11S-53W,11S-51W,11S-49W,11S-47W,11S-45W,11S-43W,11S-41W,11S-39W,11S-37W,11S-35W,11S-33W,11S-31W,11S-29W,11S-27W,11S-25W,11S-23W,11S-21W,11S-19W,11S-17W,11S-15W,11S-13W,11S-11W,11S-9W,11S-7W,11S-5W,11S-3W,11S-1W,11S-1E,11S-3E,11S-5E,11S-7E,11S-9E,11S-11E,11S-13E,11S-15E,13S-59W,13S-57W,13S-55W,13S-53W,13S-51W,13S-49W,13S-47W,13S-45W,13S-43W,13S-41W,13S-39W,13S-37W,13S-35W,13S-33W,13S-31W,13S-29W,13S-27W,13S-25W,13S-23W,13S-21W,13S-19W,13S-17W,13S-15W,13S-13W,13S-11W,13S-9W,13S-7W,13S-5W,13S-3W,13S-1W,13S-1E,13S-3E,13S-5E,13S-7E,13S-9E,13S-11E,13S-13E,13S-15E,15S-59W,15S-57W,15S-55W,15S-53W,15S-51W,15S-49W,15S-47W,15S-45W,15S-43W,15S-41W,15S-39W,15S-37W,15S-35W,15S-33W,15S-31W,15S-29W,15S-27W,15S-25W,15S-23W,15S-21W,15S-19W,15S-17W,15S-15W,15S-13W,15S-11W,15S-9W,15S-7W,15S-5W,15S-3W,15S-1W,15S-1E,15S-3E,15S-5E,15S-7E,15S-9E,15S-11E,15S-13E,15S-15E,17S-59W,17S-57W,17S-55W,17S-53W,17S-51W,17S-49W,17S-47W,17S-45W,17S-43W,17S-41W,17S-39W,17S-37W,17S-35W,17S-33W,17S-31W,17S-29W,17S-27W,17S-25W,17S-23W,17S-21W,17S-19W,17S-17W,17S-15W,17S-13W,17S-11W,17S-9W,17S-7W,17S-5W,17S-3W,17S-1W,17S-1E,17S-3E,17S-5E,17S-7E,17S-9E,17S-11E,17S-13E,17S-15E,19S-59W,19S-57W,19S-55W,19S-53W,19S-51W,19S-49W,19S-47W,19S-45W,19S-43W,19S-41W,19S-39W,19S-37W,19S-35W,19S-33W,19S-31W,19S-29W,19S-27W,19S-25W,19S-23W,19S-21W,19S-19W,19S-17W,19S-15W,19S-13W,19S-11W,19S-9W,19S-7W,19S-5W,19S-3W,19S-1W,19S-1E,19S-3E,19S-5E,19S-7E,19S-9E,19S-11E,19S-13E,19S-15E
1964-07-01,NaN,NaN,NaN,NaN,9.800510,10.692988,11.229426,11.280514,12.702756,14.402778,15.250246,17.755281,20.329781,21.781644,20.930361,20.497805,22.013178,24.960970,32.221887,39.658164,44.851979,43.793721,38.896015,33.003788,28.171084,24.946743,20.969025,17.910053,15.373028,16.347783,20.360010,25.279438,29.320471,30.852553,31.047383,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.454773,17.288725,19.159593,22.380572,24.177055,25.437178,26.802985,25.768974,23.726146,20.942302,19.523319,21.

## Preprossessing data

In [84]:
### Removing NaN

In [85]:
atn_sst_df.dropna(axis=1, how='any',inplace=True)
ats_sst_df.dropna(axis=1, how='any', inplace=True)
tn_sst_df.dropna(axis=1, how='any', inplace=True)
ts_sst_df.dropna(axis=1, how='any', inplace=True)

atn_pws_df.dropna(axis=1, how='any', inplace=True)
ats_pws_df.dropna(axis=1, how='any', inplace=True)
tn_pws_df.dropna(axis=1, how='any', inplace=True)
ts_pws_df.dropna(axis=1, how='any', inplace=True)

In [86]:
## TODO: feature scaling

In [87]:
scaler = MinMaxScaler()

atn_sst_scaled_df = pd.DataFrame(scaler.fit_transform(atn_sst_df), columns=atn_sst_df.columns)
atn_sst_scaled_df.index = atn_sst_df.index

ats_sst_scaled_df = pd.DataFrame(scaler.fit_transform(ats_sst_df), columns=ats_sst_df.columns)
ats_sst_scaled_df.index = ats_sst_df.index

tn_sst_scaled_df = pd.DataFrame(scaler.fit_transform(tn_sst_df), columns=tn_sst_df.columns)
tn_sst_scaled_df.index = tn_sst_df.index

ts_sst_scaled_df = pd.DataFrame(scaler.fit_transform(ts_sst_df), columns=ts_sst_df.columns)
ts_sst_scaled_df.index = ts_sst_df.index

atn_pws_scaled_df = pd.DataFrame(scaler.fit_transform(atn_pws_df), columns=atn_pws_df.columns)
atn_pws_scaled_df.index = atn_pws_df.index

ats_pws_scaled_df = pd.DataFrame(scaler.fit_transform(ats_pws_df), columns=ats_pws_df.columns)
ats_pws_scaled_df.index = ats_pws_df.index

tn_pws_scaled_df = pd.DataFrame(scaler.fit_transform(tn_pws_df), columns=tn_pws_df.columns)
tn_pws_scaled_df.index = tn_pws_df.index

ts_pws_scaled_df = pd.DataFrame(scaler.fit_transform(ts_pws_df), columns=ts_pws_df.columns)
ts_pws_scaled_df.index = tn_pws_df.index

### Creating transformed df for ATN,ATS,TN,TS